In [1]:
import csv
import random
import os
import glob

In [13]:
path = "G:/Downloads/Data/train"
path_up = "G:/Downloads/Data"

In [21]:
x = os.listdir(path)

y = []
random_ten_percent = []

i=0

for file in x:
    if "green" in file:
        y.append(file)


seed = 1
random.seed=seed

for file in y:
    number = random.randint(1,10)
    if number == 1:
        random_ten_percent.append(file)
        
# print(os.getcwd())

os.chdir(path_up)

with open('train.csv', 'r') as d:
    reader = csv.reader(d)
    train_data = list(reader)
     



In [29]:
print(train_data[1][0])

test = train_data[1][0]

output = []

j = 0
for file in y:
    for point in train_data:
        if point[0] in file:
            output.append([file, point[1]])
            


00070df0-bbc3-11e8-b2bc-ac1f6b6435d0


In [32]:
with open('dataset.csv', 'w') as f:
    for line in output:
        f.write(line[0])
        f.write(",")
        f.write(line[1])
        f.write('\n')